In [8]:
class IncorrectTypeException(Exception):
    def __init__(self, variable_name, expected_type, actual_type):
        message = f'Expected type of {variable_name} to be {expected_type}' \
                      f', but it was actually of type {actual_type}.'
        super(IncorrectTypeException, self).__init__(message)

In [4]:
class Directions:
    '''
    Simple, up down right left 2D directions.
    '''
    right = 'r'
    left = 'l'
    up = 'u'
    down = 'd'

In [9]:
class _Direction:
    '''
    Stores the horizontal and vertical components of a direction.
    '''
    def __init__(self, quadrant):
        '''
        parameters
            quadrant: the cartesian quadrant that the pixel
                      directed toward
        summary
            sets self.horizontal and self.vertical 
        '''
        if quadrant not in (1, 2, 3, 4):
            raise Exception('Not a valid quadrant.')
        elif quadrant == 1:
            self.horizontal, self.vertical = Directions.Right, Directions.Up
        elif quadrant == 2:
            self.horizontal, self.vertical = Directions.Left, Directions.Up
        elif quadrant == 3:
            self.horizontal, self.vertical = Directions.Left, Directions.Down
        else:
            self.horizontal, self.vertical = Directions.Right, Directions.Down

In [6]:
class DirectionTypes:
    Horizontal = 'horizontal'
    Vertical = 'vertical'

In [7]:
class PixelLocation:
    def __init__(self):
        ''' Basic constructor, no parameters'''
        self._pixel_directions = [] # will be a list of _Direction objects
        pass
    
    def add_pixel_direction(self, quadrant):
        ''' 
        Constructs a _Direction object and appends it to the
        _pixel_directions list.
        '''
        self._pixel_directions.append(_Direction(quadrant))
        
    def _get_num_pixels(self):
        ''' Self explanatory. Returned variable is an integer.'''
        return 4**len(self._pixel_directions)
    
    @staticmethod
    def _directory_to_binary(direction_type, directions):
        '''
        parameters
            direction_type: specifies either horizontal or vertical
            directions: list of either horizontal or vertical directions
        returns
            binary string that represents the index for directions
            '''
        valid_direction_types = (DirectionTypes.Horizontal, DirectionTypes.Vertical)
        if direction_type not in valid_direction_types:
            raise Exception('Invalid direction_type argument;'
                            + ' should have been one of the following:'
                            + f'\n{valid_direction_types}')
        
        if direction_type == DirectionTypes.Horizontal:
            direction_to_binary = {Directions.left : 0, Directions.right : 1}
            valid_directions = (Directions.left, Directions.right)
        else:
            direction_to_binary = {Directions.up : 0, Directions.down : 1}
            valid_directions = (Directions.down, Directions.up)
        
        binary_str = ''
        for direction in directions:
            if direction in valid_directions:
                binary_str += direction_to_binary[direction]
            else:
                raise Exception('Invalid value in directions. Expected direction to be in'
                               + valid_directions + f', but value was{direction}')
        
        return binary_str    
    
    def get_pixel_location(self):
        '''
        Given the current pixel directions,
        find the location of the pixel in the
        image. Keep in mind, the number of
        specified directions indicates the size
        of the image.
        
        returns: tuple (i, j), where if the image were
                 a 2D array of pixels, the pixel would
                 be in the ith row and jth column. This
                 is standard array convention, NOT image
                 convention (column x row).
        '''
        # First, get the vertical and horizontal directories
        horiz_dir = [direction.horizontal for direction in self._pixel_directions]
        vert_dir = [direction.vertical for direction in self._pixel_directions]
        
        # Now, get the binary string representing horizontal and vertical indices
        horiz_index_binary = _directory_to_binary(DirectionTypes.Horizontal, horiz_dir)
        vert_index_binary = _directory_to_binary(DirectionTypes.Vertical, vert_dir)
        
        # Now, convert binary to decimal
        binary = 2
        i, j = int(horiz_index_binary, binary), int(vert_index_binary, binary)
        
        return (i, j)

In [3]:
from BalancedClusters import BalancedClusters

class ImageCreation:
    def __init__(self, image_side_len, num_channels):
        self.image_side_len = image_side_len
        self.num_channels = num_channels
        self._feature_name_image = _make_blank_image_array(
            type(''), image_side_len, num_channels)
        return

    @staticmethod
    def _make_blank_image_array(content_type, image_side_len, num_channels):
        if content_type == type(str()):
            return np.array( [ [ ['' for k in range(num_channels)]
                                    for j in range(image_side_len)] 
                                  for i in range(image_side_len)] )
        elif type(content_type) != type(type('')):
            raise IncorrectTypeException('content_type', type(type('')), type(content_type))
        else:
            raise Exception('Parameter \"content_type\" was not recognized.')
            
    def _populate_pixel_with_feature_names(self, location, x_transpose_df):
        ''' 
        parameters
            location: a PixelLocation object, describing where the pixel is located
            x_transpose_df: a pandas.DataFrame, with features as examples, and examples
                            as columns
        return
            None
        summary
            Puts feature names into their proper place in the image. However,
            there is not any order to it (i.e. feature names can be placed
            in any order, as opposed to going in alphabetical order, level, etc.).
        '''
        pixel_i, pixel_j = location.get_pixel_location()
        for k, feature_name in enumerate(x_transpose_df.index.values):
            this._feature_name_image[pixel_i][pixel_j][k] = feature_name
    
    @staticmethod
    def _num_examples(df):
        ''' 
        Number of examples is the number of rows in a regular
        array, and thus the number of columns in a transposed array.
        parameters
            df: a dataframe
        returns
            integer denoting the number of examples
        '''
        actual_type, df_type = type(transposed_arr), type(pd.DataFrame())
        
        if  actual_type == df_type :
            return transposed_arr.shape[0]
        else:
            raise Exception(f'Expected transposed_arr to be of type {df_type}, but '
                           + f'the value passed was of type {actual_type}')            

    def populate_image_with_feature_names(self, x_transpose_df, location_descriptor=[]):
        '''
        parameters
            x_transpose_df:
            location_description:
            
        returns
            Nothing, just populates self._feature_name_image'''
        if _num_examples(x_transpose_df) == self.num_channels:
            # populate the three channels at the specified location in the picture
            _populate_pixel(location_descriptor, x_transpose_df)
        else:
            # get balanced clusters for each quadrant
            # must be a dictionary from name to 2D numpy array
            clusters = make_clusters(data=x_transpose_df, num_clusters=4) 
            max_iterations = 1000 # should be given by other means, but unclear right now
            cluster_balancer = BalancedClusters(clusters,'optimal')
            balanced_clusters = cluster_balancer.balance_clusters(
                max_iterations=max_iterations, verbose='none')

            

            assert len(quadrant_locations) == len(balanced_clusters.keys()) == 4
            # call fn recursively on each quadrant
            for i, key in enumerate(balanced_clusters.keys()): 
                quadrant = i+1 # quadrants begin indexing at 1
                populate_image_with_feature_names(balanced_clusters[key], 
                                                  quadrant_locations[i])